In [8]:
import numpy as np
import tensorflow as tf
from data_loader.data_loader import convert_class, data_load
from data_loader.display import create_mask, show_predictions
from IPython.display import clear_output
from model.pspunet import pspunet
from model.fcn import fcn
from model.pspunet_vgg16 import pspunet_vgg16

from model.unet import unet
import datetime
import time
from IPython.display import clear_output

IMG_WIDTH = 480
IMG_HEIGHT = 272
n_classes = 7

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
       gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
    except RuntimeError as e:
        print(e)
        
        
model = fcn((IMG_HEIGHT, IMG_WIDTH,3), n_classes)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 17, 30, 512), (None, 16, 30, 512)]

In [2]:
train_dataset, test_dataset = data_load()

In [3]:
optimizer = tf.keras.optimizers.Adam(1e-4)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

train_mIoU =tf.keras.metrics.MeanIoU(num_classes = n_classes, name = "train_mIoU")
test_mIoU =tf.keras.metrics.MeanIoU(num_classes = n_classes, name = "test_mIoU")

In [4]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/fcn_log/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/fcn_log/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [5]:
@tf.function
def train_step(images, label):
    
    with tf.GradientTape() as tape:
        pred_img = model(images)        
        loss =  loss_object(label, pred_img)

    gradients_of_model = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients_of_model, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, pred_img)
 
@tf.function
def test_step(images, label):
    pred_img = model(images)
    loss =  loss_object(label, pred_img)
    
    #pred_mask = tf.argmax(pred_img, axis=-1)
    #pred_mask = pred_mask[..., tf.newaxis]
    
    #test_mIoU(label, pred_mask)
    test_loss(loss)
    test_accuracy(label, pred_img)

In [6]:
def train(train_dataset, test_dataset, epochs, batch_size):
    
    for epoch in range(epochs):
        
        if epoch >=10:
            optimizer = tf.keras.optimizers.Adam(1e-5)

        
        start = time.time()

        train_loss.reset_states()
        train_accuracy.reset_states()    
        
        test_loss.reset_states()
        test_accuracy.reset_states()
                
        count_img = 0
        batch_time = time.time()
        
        for image_batch, label_batch in train_dataset.batch(batch_size):
            count_img += batch_size
            label_batch = convert_class(label_batch.numpy())
            
            if tf.random.uniform(()) > 0.5:
                image_batch = tf.image.flip_left_right(image_batch)
                label_batch = tf.image.flip_left_right(label_batch)

            image_batch = tf.image.random_brightness(image_batch, 0.3)
            
            train_step(image_batch, label_batch)
            
            if count_img % 1000 == 0:
                clear_output(wait=True)
                show_predictions(image_batch[:3], label_batch[:3], model)      
                print('epoch {}, step {}, train_acc {}, loss {} , time {}'.format(epoch+1,
                                                                        count_img,
                                                                        train_accuracy.result()*100,
                                                                        train_loss.result(),
                                                                        time.time()- batch_time))
                train_loss.reset_states()
                train_accuracy.reset_states()    
                
                batch_time = time.time()
                
                
        count_img = 0
        batch_time = time.time()

        for image_batch, label_batch in test_dataset.batch(batch_size):
            count_img += batch_size
            label_batch = convert_class(label_batch.numpy())
            
            test_step(image_batch, label_batch)
            
            
            if count_img % 1000 == 0:
                clear_output(wait=True)
                show_predictions(image_batch[:3], label_batch[:3], model)
                print('epoch {}, step {}, test_acc {}, loss {} , time {}'.format(epoch+1,
                                                                        count_img,
                                                                        test_accuracy.result()*100,
                                                                        test_loss.result(),
                                                                        time.time()- batch_time))
                batch_time = time.time()
                

                
        clear_output(wait=True)

        for image_batch, label_batch in test_dataset.take(3).batch(3):
            label_batch = convert_class(label_batch.numpy())
            show_predictions(image_batch[:3], label_batch[:3], model)        

        print ('Time for epoch {}  is {} sec'.format(epoch + 1, round(time.time()-start),3))

        print ('train_acc {}, loss {} , test_acc {}, loss {}'.format(train_accuracy.result()*100,
                                                                                 train_loss.result(),
                                                                                 test_accuracy.result()*100,
                                                                                 test_loss.result()
                                                                                 ))
        
        path = "model_h5/fcn/fcn_" + str(test_loss.result().numpy())+"_epoch_"+str(epoch+1)+".h5" 
        model.save(path)
            
        with train_summary_writer.as_default():
                tf.summary.scalar('loss', train_loss.result(), step=epoch+1)
                tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch+1)             
                #tf.summary.scalar('mIoU', train_mIoU.result(), step=epoch)    
                
        with test_summary_writer.as_default():
                tf.summary.scalar('loss', test_loss.result(), step=epoch+1)
                tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch+1)    
                #tf.summary.scalar('mIoU', test_mIoU.result(), step=epoch)           

In [7]:
train(train_dataset, test_dataset, 20, 8)

ValueError: in converted code:

    <ipython-input-5-404060b3a984>:5 train_step  *
        pred_img = model(images)
    /home/hyeok/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/hyeok/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:708 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    /home/hyeok/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:860 _run_internal_graph
        output_tensors = layer(computed_tensors, **kwargs)
    /home/hyeok/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/hyeok/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/layers/merge.py:182 call
        return 

In [64]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
model_18 (Model)                [(None, 16, 16, 256) 14714688    input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_170 (Conv2D)             (None, 4, 4, 4096)   102764544   model_18[1][2]                   
__________________________________________________________________________________________________
conv2d_171 (Conv2D)             (None, 4, 4, 21)     86037       conv2d_170[0][0]                 
___________________________________________________________________________________________

In [ ]:
test_loss.reset_states()
test_accuracy.reset_states()
test_mIoU.reset_states()
model = unet((IMG_HEIGHT, IMG_WIDTH,3), n_classes)
model.load_weights("model_h5/unet/unet_0.35704005_epoch_9.h5")
count_img = 0
batch_time = time.time()

for image_batch, label_batch in test_dataset.batch(8):
    count_img += 8
    label_batch = convert_class(label_batch.numpy())
    
    test_step(image_batch, label_batch)
            
    if count_img % 1000 == 0:
        clear_output(wait=True)
        show_predictions(image_batch[:3], label_batch[:3], model)
        print('epoch {}, step {}, test_acc {}, loss {} ,mIoU {}, time {}'.format(1,
                                                                count_img,
                                                                test_accuracy.result()*100,
                                                                test_loss.result(),
                                                                test_mIoU.result(),
                                                                time.time()- batch_time))
        batch_time = time.time()

In [11]:
print('epoch {}, step {}, test_acc {}, loss {} ,mIoU {}, time {}'.format(1,
                                                                count_img,
                                                                test_accuracy.result()*100,
                                                                test_loss.result(),
                                                                test_mIoU.result(),
                                                                time.time()- batch_time))

epoch 1, step 12800, test_acc 0.0, loss 0.0 ,mIoU 0.708968997001648, time 383.49503445625305


In [12]:
count_img

12800